# 2022-12-14
## Wave format data

`-` Motivation:  
Deep learning을 음성 인식이나 음성 합성과 같은 부분에 적용하고 싶어서 공부를 하게 되었다.

`-` 이 글은 누군가에게 내용을 알려주려고 하는 것이 아닌, 필자가 공부한 내용을 정리하는 용도로 작성하는 것이다.

필자가 잘못 이해하고 잘못 작성한 부분도 있을 수 있으니, 공부용으로는 적절치 못할수도 있다.

학습 내용의 출처: https://www.youtube.com/watch?v=RxbkEjV7c0o&list=PL9mhQYIlKEhem5_wrQqDtNqNcaDyFrYGN

## Before start... Why the Audio tasks is important? (Motivation)

### `-` What kind of Audio tasks exist?

Sound:
- Speech Classification & Auto-tagging (Acoustic Scene / Event Identification)
    - 게임 플레이 할 때, 시야나 지도에 없어도 주변에서 기습을 할 수 있음. 적 캐릭터의 발소리나 캐릭터 고유의 특정 소리를 잘 듣고 캐치하면 map에서 안보이는 적에 대해, 기습에 대비할 수 있는 확률이 늘어날 것임.

Speech:
- Speech Recognition (STT): 음성인식
    - 누군가 발화를 했을 때, 그 발화에 대한 정보를 텍스트로 표현할 수 있는지
- Speech Synthesis (TTS): 텍스트를 넣으면, Speech로 합석해서 주는 시스템
    - ex: 어떤 연예인의 목소리로 텍스트(ex: 책)를 읽어주도록 하는 것.
- Speech Style Transfer (STS): Speech to Speech. (음성의 스타일을 바꾸는 것)
    - ex: 사투리를 표준어로, 혹은 표준어를 사투리로. 
    - ex: 내 목소리로 성공적인 고음 노래를 부르면 어떻게 될지.

`-` 최근 Audio task관련해서 나온 성공적인 제품으로는 스마트 스피커가 있다.

## Let's get start

`-` 소리란 무엇인가?  
소리는 파동을 의미한다. 말을 하거나 물체끼리 부딪히면 소리가 나는데, 이것은 파동이 우리의 귀에 전달되어서 소리가 들리게 되는 것이다. 

우리에게 도달하는 파동(소리)는 연속적이다. 하지만, 컴퓨터는 연속적으로 이를 반영할 수 없다. (동영상에서 프레임의 개념을 떠올려보면 비슷한 맥락으로 생각할 수 있다.)  
컴퓨터가 이를 받아들이려면 이산화를 해서 받아들여야 한다. 이를 위해 아날로그 신호를 표본화(Sampling), 양자화(Quantizing), 부호화(Encoding)을 해야한다.
- `Sampling (rate)`: 1초에 몇 개의 소리를 저장할 것인지를 나타낸다. 동영상에서 프레임의 수가 높아질 수록 게임 화면이던 동영상이던 더 매끄럽고 정확하게 나타나게 된다. 비슷한 문맥으로, 필자가 아직 오디오를 잘 아는 것은 아니지만, Sampling값이 줄어들면 오디오를 끊기고 부자연스러운 소리가 나지 않을까 추측한다. 
    - Sampling rate가 높으면 좋은 것일까? $\rightarrow$ Sampling rate가 높으면, 더 많은 정보로 소리가 저장되니까 좋긴 할텐데, 연산 효율이 그렇게 좋진 않다고 나왔다. 그래서 Sampling Rate 설정 기준은 Maximum frquency의 절반 정도로 한다 하신다 하셨는데, 이 부분은 필자는 잘 와닿진 않고, open dataset에서 자주 보이는 Sampling rate를 사용하면 된다 하신부분을 참고하면 될듯 하다. (보통 16kHz가 많이 있는 듯 하다.)
    
    - 강의에서 질문중에, 16k와 8k를 인지적으로 차이를 잘 못느끼는데, 이것이 학습에 있어선 큰 차이를 만드는가 라는 질문이였다. $\rightarrow$ 강사님께서도 두 데이터 상에서 큰 차이는 없는데, 학습 시간 차이는 크게 없으면서 성능에서 약간 더 좋아지는 효과를 보셨다 한다.

- `Quantizing`: 이 소리들은 결국 컴퓨터 데이터로 나타나면 어떤 자료형으로 표현될텐데, 이게 float64, int32 등등으로 표현될 수 있다. Amplitude를 이상적인 구간으로 나누고 이걸 반올림해서 쓰는 경향이 있다함. 만약 엄청 큰 scale의 volume의 데이터 셋을 구축했는데 처리에 시간이 오래걸리면, Sampling rate를 낮추거나, Quantizing을 해서 더 가볍게 모형을 만드는 것도 중요하다고 하심.  
    - 얼마나 많은 종류의 소리로 표현할지, **정도로 필자는 이해하고 있다.** 3 bit로 quantizing을 한다면 [000], [001], [010], [011], [100], [101], [110], [111] 정도의 음만을 표현할 수 있을 것이다. 2^3 개만의 음을 표현할 수 있을 것이다. (https://suyeon96.tistory.com/13)
    
    - 보통 quantization은 안한다 하심. (https://youtu.be/RxbkEjV7c0o?list=PL9mhQYIlKEhem5_wrQqDtNqNcaDyFrYGN&t=1468)  다만, 큰 데이터를 다룰 때는 필요할 수도 있다.
    
    - Quantization을 하면 기계음이 섞이기 시작한다. https://youtu.be/RxbkEjV7c0o?list=PL9mhQYIlKEhem5_wrQqDtNqNcaDyFrYGN&t=1530

`-` 여태까지 실습과 본 것은 1초 안의 Amplitude 값을 건드리면서 1초동안의 소리 데이터에 따른 실제 들리는 오디오의 변화였다.  
1초 안에서의 Amplitude 말고 들리는 소리에 영향을 주는 **다른 요소**는 무엇이 있을까?

먼저 인간이 소리를 어떻게 인지하는지 부터 알아보자.

`-` What is wave?  
Sound: 진동으로 인한 공기의 압축  
Wave: 압축이 얼마나 됬는지 -> 압축이 얼마나 됬느냐에 따라 소리가 달라진다.

소리에서 얻을 수 있는 물리량:
- Amplitude(Intensity): 진폭  
- Frequency: 주파수 (1초 동안 얼마나 반복되는지) $\rightarrow$ 소리의 높낮이를 결정한다.
- Phase(Degree of displacement): 위상

`-` Complex Wave:  
우리가 듣는 소리들은 단일한 frequency가 존재하는 소리가 아닌, 서로 다른 정현파들의 합으로 이루어진 복합파이다.
- 정형파: 사인함수의 기하학적 그래프로, 일종의 복소 주기함수라고 보면 된다고 하신다.